In [1]:
import numpy as np
import pandas as pd

In [3]:
final_df = pd.read_csv('/kaggle/input/finale-dataset/train_dataset.csv')
test_df = pd.read_csv('/kaggle/input/finale-dataset/test_dataset.csv')

In [4]:
final_df.head()

,image_id,question,answer,path
0,71BN3iMoGkL,What are the figurines sitting on?,Bench,9d/9dd2d3f1.jpg
1,71AH7yyOgCL,What brand is the mobile phone case?,Solimo,f3/f3ab1fbf.jpg
2,71ag2dwtVfL,What brand is this?,AmazonBasics,62/62075c07.jpg
3,61yqMXIGHVL,What lining do these envelopes have?,Bubble,2d/2d483c93.jpg
4,71d57C76BpL,What is the shade shape?,Drum,b8/b8b510a2.jpg


In [5]:
# final_df= final_df.head(38000)

In [6]:
len(final_df)

63639

In [6]:
len(test_df)

15910

In [ ]:
!pip install transformers accelerate timm

In [ ]:
import pandas as pd

csv_path = "/kaggle/input/vr-dataset/images.csv"
df = pd.read_csv(csv_path)
print(df.head())

In [ ]:
image_id_to_path = dict(zip(df['image_id'], df['path']))

In [ ]:
import json


with open('/kaggle/input/vr-dataset/merge_prepro_t.json', 'r') as f:
    qa_data = json.load(f)

In [ ]:
qa_data

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BLIP-2 (flan-t5-xl is a good checkpoint for Q&A tasks)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16).to(device)



In [ ]:
from torch.utils.data import DataLoader
import numpy as np

# Parameters
BATCH_SIZE = 8  # You can experiment with 8, 16, or 32 depending on GPU memory

batched_images = []
batched_questions = []
references = []

for entry in qa_data:
    image_id = entry["image_id"]
    image_path = f"/kaggle/input/vr-dataset/small/small/{image_id_to_path[image_id]}"
    image = Image.open(image_path).convert("RGB")

    for qa in entry["questions"]:
        question = qa["question"]
        true_answer = qa["answer"]

        batched_images.append(image)
        batched_questions.append(question)
        references.append(true_answer)

print(f"Total QA pairs: {len(batched_questions)}")

# Now process in batches
predictions = []

for i in range(0, len(batched_questions), BATCH_SIZE):
    batch_imgs = batched_images[i:i+BATCH_SIZE]
    batch_qs = batched_questions[i:i+BATCH_SIZE]

    inputs = processor(images=batch_imgs, text=batch_qs, return_tensors="pt", padding=True).to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=10)

    batch_outputs = processor.batch_decode(generated_ids, skip_special_tokens=True)
    print(batch_outputs)
    predictions.extend([ans.strip() for ans in batch_outputs])

print("done batching")

In [ ]:
!pip install bert-score 
from bert_score import score
from sklearn.metrics import accuracy_score

# BERTScore
P, R, F1 = score(predictions, references, lang="en", verbose=True)

# Exact match (case-insensitive single-word comparison)
exact_match = [pred.lower() == ref.lower() for pred, ref in zip(predictions, references)]

# Print results
print(f"Average BERTScore F1: {F1.mean().item():.4f}")
print(f"Exact Match Accuracy: {sum(exact_match) / len(exact_match):.4f}")

# LORA - MODEL

In [ ]:
final_df.columns

## Create PyTorch DataSet

In [ ]:
import torch
from torch.utils.data import Subset,DataLoader
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset,load_dataset
from PIL import Image
import pandas as pd
from torchvision import transforms



In [ ]:
train_dataset = torch.load("/kaggle/input/tensor-data/train_batch_2.pt", weights_only=True)
print("Train Dataset created....")
# train_batch_2 = torch.load("/kaggle/input/tensor-data/train_batch_2.pt", weights_only=True)
test_dataset = torch.load("/kaggle/input/tensor-data/test_batch.pt", weights_only=True)
print("Test Dataset created....")

# Combine both datasets
# train_dataset = train_batch_1 + train_batch_2



In [ ]:
print(train_dataset[0])
print(len(train_dataset))

In [ ]:
print(test_dataset[0])
print(len(test_dataset))

In [ ]:
def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch])
    input_ids = torch.nn.utils.rnn.pad_sequence(
        [x["input_ids"] for x in batch], batch_first=True, padding_value=processor.tokenizer.pad_token_id
    )
    attention_mask = torch.nn.utils.rnn.pad_sequence(
        [x["attention_mask"] for x in batch], batch_first=True, padding_value=0
    )
    labels = torch.nn.utils.rnn.pad_sequence(
        [x["labels"] for x in batch], batch_first=True, padding_value=-100
    )
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

#try with lower batch size if memory gets full
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=4)

In [ ]:
from transformers import Blip2ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16)
model.to(device)
# right after you load the model:
_orig_forward = model.forward

def _forward_no_inputs_embeds(*args, **kwargs):
    kwargs.pop("inputs_embeds", None)
    kwargs.pop("decoder_inputs_embeds", None)  # Use `None` as default
    return _orig_forward(*args, **kwargs)


model.forward = _forward_no_inputs_embeds

lora_config = LoraConfig(
    r=128,  # Rank of the low-rank matrices
    lora_alpha=256,  # Scaling factor
    target_modules=["q", "v"],  # Target attention layers
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify the number of trainable parameters

In [ ]:
from transformers import get_scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scaler = torch.cuda.amp.GradScaler()  # for mixed precision
num_epochs=3


# Create the scheduler (assuming num_epochs and train_dataloader are defined)
# num_train_steps = len(train_dataloader) * num_epochs  # Total training steps
num_train_steps = 1000
warmup_steps = int(num_train_steps * 0.1)  # Warm-up for 10% of total steps

# Set up the learning rate scheduler
lr_scheduler = get_scheduler(
    "cosine",  # Linear warm-up followed by linear decay
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_train_steps
)
# new part added

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Assume batch is a dict with keys 'pixel_values', 'input_ids', 'attention_mask', 'labels'
        pixel_values = batch["pixel_values"].to(device, torch.float16)        # image tensor
        input_ids = batch["input_ids"].to(device)                            # text input IDs
        attention_mask = batch["attention_mask"].to(device)                  # text mask
        labels = batch["labels"].to(device)                                  # target IDs (with -100 as needed)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():  # mixed precision context
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        #this one line is the only change
       # Update the learning rate with the scheduler
        lr_scheduler.step()  # This updates the learning rate after each batch
        
    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in test_dataloader:
            pixel_values = batch["pixel_values"].to(device, torch.float16)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            val_loss += outputs.loss.item()
    val_loss /= len(test_dataloader)
    print(f"Epoch {epoch}: validation loss = {val_loss}")



In [ ]:

!pip install bert-score
from bert_score import score

model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in test_dataloader:
        pixel_values = batch["pixel_values"].to(device, torch.float16)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        generated_ids = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20  # Adjust as needed
        )

        # Decode generated predictions
        generated_texts = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend(generated_texts)

        # Decode ground-truth labels
        label_ids = batch["labels"]
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id  # Replace -100 with pad_token_id
        true_texts = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
        references.extend(true_texts)


In [ ]:
P, R, F1 = score(predictions, references, lang="en", verbose=True)

print(f"Average BERTScore F1: {F1.mean().item():.4f}")


In [ ]:
predictions

In [ ]:
references

# LORA MODEL TRAINING 

In [ ]:
!pip install -q peft accelerate


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from transformers import BlipProcessor, BlipForQuestionAnswering, get_scheduler
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base"
).to(device)

_orig_forward = model.forward

def _forward_no_inputs_embeds(*args, **kwargs):
    kwargs.pop("inputs_embeds", None)
    kwargs.pop("decoder_inputs_embeds", None)  # Use `None` as default
    return _orig_forward(*args, **kwargs)

model.forward = _forward_no_inputs_embeds

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

df = pd.read_csv("/kaggle/input/finale-dataset/train_dataset.csv")  # assumes columns: 'path', 'question', 'answer'

class BlipVQADataset(Dataset):
    def __init__(self, df, image_dir, processor):
        self.df = df
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row['path'])
        image = Image.open(image_path).convert("RGB")
        question = str(row["question"])
        answer = str(row["answer"])

        inputs = self.processor(images=image, text=question, return_tensors="pt", padding='max_length', max_length=128, truncation=True)
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = processor.tokenizer(answer, return_tensors="pt", padding='max_length', max_length=32, truncation=True).input_ids.squeeze(0)
        return inputs

def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch], batch_first=True, padding_value=-100
    )
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

image_dir = "/kaggle/input/vr-dataset/small/small/"
dataset = BlipVQADataset(df, image_dir, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=2)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler("cosine", optimizer=optimizer, num_warmup_steps=100, num_training_steps=len(dataloader)*3)

scaler = torch.cuda.amp.GradScaler()

model.train()
for epoch in range(3):
    total_loss = 0
    for step, batch in enumerate(dataloader):
        pixel_values = batch["pixel_values"].to(device, dtype=torch.float16)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            # Correct forward pass without 'inputs_embeds'
            outputs = model(
                pixel_values=pixel_values,  # Image features
                input_ids=input_ids,        # Text input (question)
                attention_mask=attention_mask,  # Attention mask
                labels=labels               # Labels for the answer
            )
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()

        total_loss += loss.item()
        if step % 10 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} complete. Avg Loss: {total_loss / len(dataloader):.4f}")

save_path = "/kaggle/working/blip_lora_adapter"
model.save_pretrained(save_path)
print(f"LoRA adapter saved at: {save_path}")


In [ ]:
class BlipVQATestDataset(Dataset):
    def __init__(self, df, image_dir, processor):
        self.df = df
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row['path'])
        image = Image.open(image_path).convert("RGB")
        question = str(row["question"])

        inputs = self.processor(images=image, text=question, return_tensors="pt", padding='max_length', max_length=128, truncation=True)
        return {k: v.squeeze(0) for k, v in inputs.items()}
test_dataset = BlipVQATestDataset(test_df, image_dir="/kaggle/input/vr-dataset/small/small/", processor=processor)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=5
        )

        answers = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(answers)
test_df["predicted_answer"] = predictions
test_df.head()

In [ ]:
from sklearn.metrics import accuracy_score
from bert_score import score

test_df["Answer_clean"] = test_df["answer"].astype(str).str.lower().str.strip()
test_df["Pred_clean"] = test_df["predicted_answer"].astype(str).str.lower().str.strip()

acc = accuracy_score(test_df["Answer_clean"], test_df["Pred_clean"])
print(f"Exact Match Accuracy: {acc:.4f}")

filtered_df = test_df[(test_df["Answer_clean"] != "") & (test_df["Pred_clean"] != "")]

P, R, F1 = score(
    filtered_df["Pred_clean"].tolist(),
    filtered_df["Answer_clean"].tolist(),
    lang="en",
    verbose=True
)
print(f"BERTScore - P: {P.mean():.4f}, R: {R.mean():.4f}, F1: {F1.mean():.4f}")


In [ ]:
save_path = "/kaggle/working/blip_lora_adapter"
model.save_pretrained(save_path)
print(f"LoRA adapter saved at: {save_path}")

In [ ]:
!zip -r /kaggle/working/blip_lora_adapter.zip /kaggle/working/blip_lora_adapter


# Flash

In [ ]:
!pip install -q peft transformers accelerate timm


In [ ]:
from transformers import BlipForQuestionAnswering, BlipProcessor

base_model_name = "Salesforce/blip-vqa-base"
processor = BlipProcessor.from_pretrained(base_model_name)
model = BlipForQuestionAnswering.from_pretrained(base_model_name)


In [ ]:
from peft import PeftModel

adapter_path = "/kaggle/input/lora-model/adapter_model.safetensors"
model = PeftModel.from_pretrained(model, adapter_path)


# BASE-MODEL

In [21]:
!pip install transformers torch torchvision pillow


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score
from bert_score import score

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model & processor
model_name = "Salesforce/blip-vqa-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name).to(device)
model.eval()

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [32]:
class VQADataset(Dataset):
    def __init__(self, df, image_dir, processor):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["path"])
        image = Image.open(img_path).convert("RGB")
        question = row["question"]

        inputs = self.processor(
            image,
            question,
            return_tensors="pt",
            padding="max_length",
            max_length=128,
            truncation=True
        )
        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "answer": row["answer"]
        }

# Custom collate function
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    answers = [item["answer"] for item in batch]
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "answer": answers
    }



image_dir = "/kaggle/input/vr-dataset/small/small" 

# Prepare DataLoader
test_dataset = VQADataset(test_df, image_dir, processor)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)


In [33]:
predictions = []
true_answers = []

with torch.no_grad():
    for batch in test_loader:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10,
            use_cache=True
        )

        preds = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(preds)
        true_answers.extend(batch["answer"])

In [30]:
!pip install bert-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [34]:
test_df["predicted_answer"] = predictions
test_df["Answer_clean"] = test_df["answer"].astype(str).str.lower().str.strip()
test_df["Pred_clean"] = test_df["predicted_answer"].astype(str).str.lower().str.strip()

acc = accuracy_score(test_df["Answer_clean"], test_df["Pred_clean"])
print(f"Exact Match Accuracy: {acc:.4f}")


Exact Match Accuracy: 0.1725


In [35]:
filtered_df = test_df[(test_df["Answer_clean"] != "") & (test_df["Pred_clean"] != "")]

P, R, F1 = score(
    filtered_df["Pred_clean"].tolist(),
    filtered_df["Answer_clean"].tolist(),
    lang="en",
    verbose=True
)

print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/40 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/249 [00:00<?, ?it/s]

done in 4.73 seconds, 3363.31 sentences/sec
BERTScore - Precision: 0.9517, Recall: 0.9251, F1: 0.9371


In [37]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [38]:
from nltk.corpus import wordnet as wn

def wups_score(preds, refs, threshold=0.9):
    def compute_wups(pred, ref):
        pred_synsets = wn.synsets(pred)
        ref_synsets = wn.synsets(ref)

        if not pred_synsets or not ref_synsets:
            return 0.0  # no synsets found → assume unrelated

        max_score = max(wn.wup_similarity(p, r) or 0.0 for p in pred_synsets for r in ref_synsets)
        return max_score

    scores = []
    for pred, ref in zip(preds, refs):
        pred = pred.lower().strip()
        ref = ref.lower().strip()

        score = compute_wups(pred, ref)
        score = score if score >= threshold else 0.0  # apply threshold
        scores.append(score)

    return sum(scores) / len(scores) if scores else 0.0


In [39]:
test_df["Answer_clean"] = test_df["answer"].astype(str).str.lower().str.strip()
test_df["Pred_clean"] = test_df["predicted_answer"].astype(str).str.lower().str.strip()

refs = test_df["Answer_clean"].tolist()
preds = test_df["Pred_clean"].tolist()

wups_00 = wups_score(preds, refs, threshold=0.0)
wups_09 = wups_score(preds, refs, threshold=0.9)

final_score = 0.5 * wups_00 + 0.5 * wups_09

print(f"WUPS @0.0 (lenient): {wups_00:.4f}")
print(f"WUPS @0.9 (strict): {wups_09:.4f}")
print(f"Final Weighted WUPS Score: {final_score:.4f}")


WUPS @0.0 (lenient): 0.4538
WUPS @0.9 (strict): 0.2116
Final Weighted WUPS Score: 0.3327


In [40]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & fast

preds = test_df["predicted_answer"].astype(str).tolist()
refs = test_df["answer"].astype(str).tolist()

pred_embeds = model.encode(preds, convert_to_tensor=True)
ref_embeds = model.encode(refs, convert_to_tensor=True)

cos_sim = util.cos_sim(pred_embeds, ref_embeds).diagonal()

sbert_score = cos_sim.mean().item()
print(f"SBERT Metric: {sbert_score:.4f}")


Batches:   0%|          | 0/498 [00:00<?, ?it/s]

Batches:   0%|          | 0/498 [00:00<?, ?it/s]

SBERT Metric: 0.4584


In [41]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Tokenize and compute METEOR per pair
meteor_scores = [
    meteor_score([word_tokenize(ref)], word_tokenize(pred))
    for ref, pred in zip(test_df["answer"].astype(str), test_df["predicted_answer"].astype(str))
]

# Average score
avg_meteor = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR Score: {avg_meteor:.4f}")


Average METEOR Score: 0.0994


In [36]:
test_df.head()

,image_id,question,answer,path,predicted_answer,Answer_clean,Pred_clean
0,71OQ4BiD3uL,What is the product type?,Bucket,f0/f0d50e0a.jpg,no,bucket,no
1,81CtH3N+0NL,What kind of closure does it have?,Lace-up,52/521668c5.jpg,sliding,lace-up,sliding
2,8176aHTRP3L,What is the primary color?,Multi,43/438d87ff.jpg,blue,multi,blue
3,71yC+5KLN-L,What material is the cover made of?,Hard,30/304914ac.jpg,metal,hard,metal
4,71pP-5SEtyL,What material is the case made of?,Silicone,40/40d177f8.jpg,metal,silicone,metal


# MODEL-2

In [1]:
!pip install torch torchvision transformers peft pandas pillow



In [7]:
import os
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from peft import PeftModel
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base",
    device_map='auto'
)


model = PeftModel.from_pretrained(model, "bk45/blip-vqa-finetuned").to(device)
model.eval()

# Dataset class
class BlipVQATestDataset(Dataset):
    def __init__(self, df, image_dir, processor):
        self.df = df
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row['path'])
        image = Image.open(image_path).convert("RGB")
        question = str(row["question"])

        inputs = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding='max_length',
            max_length=128,
            truncation=True
        )
        return {k: v.squeeze(0) for k, v in inputs.items()}

# Load test dataframe & image directory
# test_df = pd.read_csv("/kaggle/input/vr-dataset/test.csv")  # Adjust path if needed
image_dir = "/kaggle/input/vr-dataset/small/small/"

# Prepare DataLoader
test_dataset = BlipVQATestDataset(test_df, image_dir, processor)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

# Inference loop
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10,
            use_cache=True
        )

        answers = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(answers)

# Save or print predictions
test_df["predicted_answer"] = predictions
test_df.head(10)

,image_id,question,answer,path,predicted_answer
0,71OQ4BiD3uL,What is the product type?,Bucket,f0/f0d50e0a.jpg,hand
1,81CtH3N+0NL,What kind of closure does it have?,Lace-up,52/521668c5.jpg,lace
2,8176aHTRP3L,What is the primary color?,Multi,43/438d87ff.jpg,pink
3,71yC+5KLN-L,What material is the cover made of?,Hard,30/304914ac.jpg,hard
4,71pP-5SEtyL,What material is the case made of?,Silicone,40/40d177f8.jpg,canvas
5,71A8O6XIQGL,What is the shoe's style?,Ballet,10/104a3774.jpg,heels
6,61NxifiMbuL,Is warranty included?,No,42/4213f453.jpg,no
7,71sqlEJ6F1L,What is the case's material?,Silicon,3c/3ce01595.jpg,canvas
8,81UrYa7KFEL,What is the heat level?,Medium,b6/b613a781.jpg,medium
9,71gGW9pDoHL,What is the shoe's style?,Sneakers,e2/e2f5d987.jpg,sneakers


In [8]:
test_df.to_csv("vqa_predictions.csv", index=False)


In [47]:
test_df = pd.read_csv("vqa_predictions.csv")

In [48]:
test_df.head()

,image_id,question,answer,path,predicted_answer
0,71OQ4BiD3uL,What is the product type?,Bucket,f0/f0d50e0a.jpg,hand
1,81CtH3N+0NL,What kind of closure does it have?,Lace-up,52/521668c5.jpg,lace
2,8176aHTRP3L,What is the primary color?,Multi,43/438d87ff.jpg,pink
3,71yC+5KLN-L,What material is the cover made of?,Hard,30/304914ac.jpg,hard
4,71pP-5SEtyL,What material is the case made of?,Silicone,40/40d177f8.jpg,canvas


In [49]:
test_df["Answer_clean"] = test_df["answer"].astype(str).str.lower().str.strip()
test_df["Pred_clean"] = test_df["predicted_answer"].astype(str).str.lower().str.strip()

acc = accuracy_score(test_df["Answer_clean"], test_df["Pred_clean"])
print(f"Exact Match Accuracy: {acc:.4f}")


Exact Match Accuracy: 0.2864


In [50]:
filtered_df = test_df[(test_df["Answer_clean"] != "") & (test_df["Pred_clean"] != "")]

P, R, F1 = score(
    filtered_df["Pred_clean"].tolist(),
    filtered_df["Answer_clean"].tolist(),
    lang="en",
    verbose=True
)

print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/39 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/249 [00:00<?, ?it/s]

done in 4.40 seconds, 3618.49 sentences/sec
BERTScore - Precision: 0.9552, Recall: 0.9344, F1: 0.9438


In [9]:
! pip install nltk



In [10]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [11]:
from nltk.corpus import wordnet as wn

def wups_score(preds, refs, threshold=0.9):
    def compute_wups(pred, ref):
        pred_synsets = wn.synsets(pred)
        ref_synsets = wn.synsets(ref)

        if not pred_synsets or not ref_synsets:
            return 0.0  # no synsets found → assume unrelated

        max_score = max(wn.wup_similarity(p, r) or 0.0 for p in pred_synsets for r in ref_synsets)
        return max_score

    scores = []
    for pred, ref in zip(preds, refs):
        pred = pred.lower().strip()
        ref = ref.lower().strip()

        score = compute_wups(pred, ref)
        score = score if score >= threshold else 0.0  # apply threshold
        scores.append(score)

    return sum(scores) / len(scores) if scores else 0.0


In [13]:
# Clean answers
test_df["Answer_clean"] = test_df["answer"].astype(str).str.lower().str.strip()
test_df["Pred_clean"] = test_df["predicted_answer"].astype(str).str.lower().str.strip()

# Get lists
refs = test_df["Answer_clean"].tolist()
preds = test_df["Pred_clean"].tolist()

# Calculate WUPS@0.0 (lenient) and WUPS@0.9 (strict)
wups_00 = wups_score(preds, refs, threshold=0.0)
wups_09 = wups_score(preds, refs, threshold=0.9)

# Optional: combine
final_score = 0.5 * wups_00 + 0.5 * wups_09

# Print results
print(f"WUPS@0.0 (lenient): {wups_00:.4f}")
print(f"WUPS@0.9 (strict): {wups_09:.4f}")
print(f"Final Weighted WUPS Score: {final_score:.4f}")


WUPS@0.0 (lenient): 0.5673
WUPS@0.9 (strict): 0.3064
Final Weighted WUPS Score: 0.4369


In [15]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & fast

preds = test_df["predicted_answer"].astype(str).tolist()
refs = test_df["answer"].astype(str).tolist()

pred_embeds = model.encode(preds, convert_to_tensor=True)
ref_embeds = model.encode(refs, convert_to_tensor=True)

cos_sim = util.cos_sim(pred_embeds, ref_embeds).diagonal()

sbert_score = cos_sim.mean().item()
print(f"SBERT Metric: {sbert_score:.4f}")


Batches:   0%|          | 0/498 [00:00<?, ?it/s]

Batches:   0%|          | 0/498 [00:00<?, ?it/s]

SBERT Metric: 0.5630


In [16]:
!pip install nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Tokenize and compute METEOR per pair
meteor_scores = [
    meteor_score([word_tokenize(ref)], word_tokenize(pred))
    for ref, pred in zip(test_df["answer"].astype(str), test_df["predicted_answer"].astype(str))
]

# Average score
avg_meteor = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR Score: {avg_meteor:.4f}")


Average METEOR Score: 0.1496
